In [1]:
%matplotlib notebook
%matplotlib inline

# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import time
import gmaps

# Google developer API key 
from config import gkey
gmaps.configure(api_key=gkey)


In [2]:
#Import csv from WeatherPy assignment
df2 = pd.read_csv('../WeatherPy/WeatherPY_Data.csv')

In [3]:
#Create a heat map that displays the humidity for every city from the part I of the homework.

# Store 'Lat' and 'Lng' into  locations 
locations = df2[['act_lat', 'act_lng']].astype(float)

# Convert Humidity to float and store
humidity = df2['Humidity_Perc'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
#Create dataframe with perferred weather conditions
df3 = df2[(df2.Temperature_F >= 70) & (df2.Temperature_F < 80) & (df2.Humidity_Perc <= 61)].copy()


In [5]:
# find first hotel in each preferred city (df2) and add to dataframe
df3['Hotel_Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'rankby': 'distance',
    'type': 'lodging',
    'key': gkey,
}

for index, row in df3.iterrows():

    # add location to params dict
    lat = row['act_lat']
    lng = row['act_lng']
    coords = (f'{lat},{lng}')
    params['location'] = coords
    
    #vars for output reporting
    city  = row['Nearest_City']

    # build response object
    response = requests.get(base_url, params=params)
    response_json = response.json()
    response_time = response.elapsed.total_seconds()

    try:
       df3.loc[index, ['Hotel_Name']] = response_json['results'][0]['name'] #hotel name
       response_time = response.elapsed.total_seconds()
       print(f'hotel response for {city} in {response_time} seconds')

      
    except (KeyError, IndexError):
        status = response_json['status']
        print(f'Response error for {city} skipping. status = {status}')
        

    time.sleep(response.elapsed.total_seconds())


hotel response for tuatapere in 0.52251 seconds
hotel response for teknaf in 0.451611 seconds
hotel response for santiago del estero in 0.48001 seconds
hotel response for puerto ayora in 0.456009 seconds
hotel response for parana in 0.441095 seconds
hotel response for opuwo in 0.390391 seconds
hotel response for nasirabad in 0.394573 seconds
hotel response for mount gambier in 0.460659 seconds
hotel response for makokou in 0.375689 seconds
hotel response for makakilo city in 0.552483 seconds
hotel response for launceston in 0.558626 seconds
hotel response for deori khas in 0.429301 seconds
hotel response for cidreira in 0.421996 seconds
Response error for boende skipping. status = ZERO_RESULTS
hotel response for bluff in 0.336549 seconds
hotel response for albany in 0.415488 seconds


In [6]:
#Create Symbol GMAP for hotels in citys with perfered weather df3

preferred_locations = df3[['act_lat', 'act_lng']].astype(float)
hotel_names = df3['Hotel_Name'].tolist()
humidity = df3['Humidity_Perc'].astype(float)

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(preferred_locations, display_info_box = True, info_box_content = hotel_names)

heat_layer = gmaps.heatmap_layer(preferred_locations, weights=humidity)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))